In [ ]:
import pandas

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

In [ ]:
#print(client.database_names())
db_twitter = client["Twitter"]
collections = db_twitter.collection_names()

In [ ]:
for i in collections:
    print(i+": {:,}".format(db_twitter[i].find({}).count()))

In [ ]:
def get_list_from_csv(data):
    ci = data.city.tolist()
    city = ci[1:]
    st = data.state.tolist()
    state = st[1:]
    co = data.country.tolist()
    country = co[1:]
    return city,state,country

# Twitter
##  1. Compute the number of tweets in Australia based on the user location
### (i) aggregate records from "twitter-australia" collection

In [ ]:
# Construct pipline for aggregation

pipeline = [
    { "$group": {
        "_id": "$user.location",
        "count": { "$sum": 1 }
        }
    }
]

In [ ]:
#  Aggregating Twitter-Australia data

aus_data = list(db_twitter['twitter-australia'].aggregate(pipeline))
print("Aggregating Twitter-Australia data is done")

### (ii) get name of city, state, country from au.csv

In [ ]:
# get list of all city name of australia

colnames = ['city', 'state', 'country']
data = pandas.read_csv('supporting-files/au.csv', names=colnames, encoding="ISO-8859-1")

In [ ]:
city, state, country = get_list_from_csv(data)
print("reading csv file of australia_cities is done")

### (iii) calculate the number of tweets based on location (Australia)

In [ ]:
aus_data_format = []

for data in aus_data:
    exist = 0
    
    for x in city:  
        loc = data["_id"].lower().replace("australia","")
        if x.lower() in loc:
            index = city.index(x)
            if len(aus_data_format)>0:
                for d in aus_data_format:
                    if x in d["id"]:
                        d["count"] += data["count"] 
                        exist = 1
                        break 
                if exist == 0:
                    aus_data_format.append({"id":x + ',' + state[index] + "," + country[index],"count":data["count"]})
                    exist = 1
            else: 
                aus_data_format.append({"id":x + ',' + state[index] + "," + country[index],"count":data["count"]})
                exist = 1
            break
    if exist == 0:
        for z in state:
            if z.lower() in data["_id"].lower():
                index = state.index(z)
                if len(aus_data_format)>0:
                    for d in aus_data_format:
                        s = d["id"].split(",")[0]
                        if (s == "null") and (z in d["id"]):
                            d["count"] += data["count"]  
                            exist = 1
                            break 
                    if exist == 0:
                        aus_data_format.append({"id":"null" + "," + z + "," + country[index],"count":data["count"]})
                        exist = 1
                else: 
                    aus_data_format.append({"id": "null" + ","  + z + "," + country[index],"count":data["count"]})  
                    exist = 1
                break
    if exist == 0:
        for y in country:
            if y.lower() in data["_id"].lower():
                if len(aus_data_format)>0:
                    for d in aus_data_format:
                        s = d["id"].split(",")[1]
                        if (s == "null") and (y in d["id"]):
                            d["count"] += data["count"]       
                            exist = 1
                            break 
                    if exist == 0:
                        aus_data_format.append({"id":"null" + "," + "null" + ',' + y,"count":data["count"]})
                        exist = 1
                else: 
                    aus_data_format.append({"id": "null" + ","  + "null" + ',' + y,"count":data["count"]}) 
                    exist = 1 
                break
                
print("list \'aus_data_format\' is finished")

### (iv) print out the output csv file

In [ ]:
# Print total number of tweets in Australia

file_name = 'output/twitter-australia-user-collection.csv'
with open(file_name, 'w') as f:
    # header
    f.write('city,state,country,Tweets from Australia\n')
    
    for data in aus_data_format:
        city = data['id'].split(",")[0]
        state = data['id'].split(",")[1]
        country = data['id'].split(",")[2]
        if state == 'Victoria':
            state = 'VIC'
        
        line = city + ',' + state + ',' +  country + ',' + str(data['count']) + '\n'
        f.write(line)
    
print (file_name, 'is ready.')

##  2. Compute the number of tweets in Other based on the user location
### (i) aggregate records from "twitter-other" collection

In [ ]:
#  Twitter-Other data

other_data = list(db_twitter['twitter-other'].aggregate(pipeline,allowDiskUse=True))
print("Aggregating Twitter-Other data is done")

### (ii) get name of city, state, country from world_cities.csv

In [ ]:
# get list of all city name around world

colnames = ['city', 'state', 'country']
data = pandas.read_csv('supporting-files/world-cities.csv', names=colnames, encoding="ISO-8859-1")

In [ ]:
city, state, country = get_list_from_csv(data)
print("reading csv file of world_cities is done")

### (iii) calculate the number of tweets based on location (World)

In [ ]:
other_data_format = []

for data in other_data:
    exist = 0
    if data["_id"] is not None: 
        for x in city: 
            if isinstance(x,str):
                if x.lower() in data["_id"].lower():
                    index = city.index(x)
                    if len(other_data_format)>0:
                        for d in other_data_format:
                            if x in d["id"]:
                                d["count"] += data["count"] 
                                exist = 1
                                break 
                        if exist == 0:
                            other_data_format.append({"id":x + ',' + str(state[index]) + "," + str(country[index]),"count":data["count"]})
                            exist = 1
                    else: 
                        other_data_format.append({"id":x + ',' + str(state[index]) + "," + str(country[index]),"count":data["count"]})
                        exist = 1
                    print("city: " + x)
                    break
        if exist == 0:
            for z in state:
                if isinstance(z, str):
                    if z.lower() in data["_id"].lower():
                        index = state.index(z)
                        if len(other_data_format)>0:
                            for d in other_data_format:
                                s = d["id"].split(",")[0]
                                if (s == "null") and (z in d["id"]):
                                    d["count"] += data["count"]  
                                    exist = 1
                                    break 
                            if exist == 0:
                                other_data_format.append({"id":"null" + "," + z + "," + str(country[index]),"count":data["count"]})
                                exist = 1
                        else: 
                            other_data_format.append({"id": "null" + ","  + z + "," + str(country[index]),"count":data["count"]})  
                            exist = 1
                        print("state: " + z)
                        break
        if exist == 0:
            for y in country:
                if isinstance(y,str):
                    if y.lower() in data["_id"].lower():
                        if len(other_data_format)>0:
                            for d in other_data_format:
                                s = d["id"].split(",")[1]
                                if (s == "null") and (y in d["id"]):
                                    d["count"] += data["count"]       
                                    exist = 1
                                    break 
                            if exist == 0:
                                other_data_format.append({"id":"null" + "," + "null" + ',' + y,"count":data["count"]})
                                exist = 1
                        else: 
                            other_data_format.append({"id": "null" + ","  + "null" + ',' + y,"count":data["count"]}) 
                            exist = 1
                        print("country: " + y)
                        break
                
print("list \'other_data_format\' is finished")

### (iv) print out the output csv file 

In [ ]:
file_name = 'output/twitter-others-user-collection.csv'
with open(file_name, 'w') as f:
    # header
    f.write('city,state,country,Tweets from Other\n')
    
    for data in other_data_format:
        city = data['id'].split(",")[0]
        state = data['id'].split(",")[1]
        country = data['id'].split(",")[2]
        
        line = city + ',' + state + ',' +  country + ',' + str(data['count']) + '\n'
        f.write(line)
    
print (file_name, 'is ready.')